In [36]:
import pandas as pd
import numpy as np
import pyodbc

In [37]:
driver = '{ODBC Driver 17 for SQL Server}'
server = '190.117.102.109,1433'  # Incluir el puerto en la dirección del servidor
database = 'siiaexplorandes'
usuario = 'consulta'
password = 'consultagerencia'
 
conexion = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={usuario};PWD={password}')

In [38]:
detalle_pi = """SELECT mov_compro,tie_razon,substring(fil_codigo,1,4) 'Año',substring(fil_codigo,5,2) 'Mes',fil_Codigo,mov_fecha 'Fecha Creac_PI',mov_fvenc 'Venc_PI',gui_docum,(replace(gui_Serie,' ','')+'-'+replace(gui_docum,' ','')) AS 'Proforma',gui_coddoc 'Tipo_PI',
sum(CASE WHEN GUI_CODDOC LIKE '%CN%' THEN -rou_submon ELSE rou_submon end) as 'Monto_PI',
CASE WHEN elimini='0' THEN 'Activo' ELSE 'Eliminado' end AS 'Estado PI'  
FROM dbo.guic2024
WHERE SUBSTRING(fil_Codigo,1,4)='2024' and (gui_coddoc<>'' OR gui_coddoc NOT LIKE '%CN%') AND ELIMINI='0' and gui_docum<>''
group by mov_compro,fil_codigo,mov_fecha,mov_fvenc,gui_coddoc,gui_serie,gui_docum,elimini,tie_razon
UNION ALL
SELECT mov_compro,tie_razon,substring(fil_codigo,1,4) 'Año',substring(fil_codigo,5,2) 'Mes',fil_Codigo,mov_fecha 'Fecha Creac_PI',mov_fvenc 'Venc_PI',gui_docum,(replace(gui_Serie,' ','')+'-'+replace(gui_docum,' ','')) AS 'Proforma',gui_coddoc 'Tipo_PI',
sum(CASE WHEN GUI_CODDOC LIKE '%CN%' THEN -rou_submon ELSE rou_submon end) as 'Monto_PI',
CASE WHEN elimini='0' THEN 'Activo' ELSE 'Eliminado' end AS 'Estado PI'  
FROM dbo.guic2023
WHERE SUBSTRING(fil_Codigo,1,4)='2024' and (gui_coddoc<>'' OR gui_coddoc NOT LIKE '%CN%') AND ELIMINI='0' and gui_docum<>''
group by mov_compro,fil_codigo,mov_fecha,mov_fvenc,gui_coddoc,gui_serie,gui_docum,elimini,tie_razon"""

detalle_pi = pd.read_sql(detalle_pi, conexion)
detalle_pi.head()


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_21296\2002773402.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  detalle_pi = pd.read_sql(detalle_pi, conexion)


,mov_compro,tie_razon,Año,Mes,fil_Codigo,Fecha Creac_PI,Venc_PI,gui_docum,Proforma,Tipo_PI,Monto_PI,Estado PI
0,00000000002,O'TOOLE x2 ...,2024,01,202401037,2024-01-03,2024-01-09,0003910,IN01-0003910,IN,9254.0,Activo
1,00000000003,PRINCE x 2 ...,2024,07,202407044,2024-01-03,2024-06-07,0003911,IN01-0003911,IN,8542.0,Activo
2,00000000004,HARPER x 5 ...,2024,07,202407045,2024-01-03,2024-06-12,0003912,IN01-0003912,IN,13523.0,Activo
3,00000000005,PRE-PAYMENTS 2024 ...,2024,01,202401039,2024-01-03,2023-12-11,0003913,IN01-0003913,IN,6680.0,Activo
4,00000000007,MINI TRIP CUSCO - SEPTEMBER x08 ...,2024,09,202409044,2024-01-04,2024-08-08,0003914,IN01-0003914,IN,12696.0,Activo


In [39]:
agrupado = detalle_pi.groupby(['Mes','fil_Codigo'])['Monto_PI'].sum().reset_index()
agrupado = agrupado[agrupado['Monto_PI']  > 0]
agrupado.head()

,Mes,fil_Codigo,Monto_PI
0,01,202401001,7590.0
1,01,202401002,24278.0
2,01,202401003,7210.0
3,01,202401004,7744.0
4,01,202401005,6102.0


In [40]:
tabla1 = agrupado.groupby(['Mes','fil_Codigo'])['Monto_PI'].sum().reset_index()
# tabla1 = tabla1[tabla1['Mes'] == '12']
tabla1

,Mes,fil_Codigo,Monto_PI
0,01,202401001,7590.0
1,01,202401002,24278.0
2,01,202401003,7210.0
3,01,202401004,7744.0
4,01,202401005,6102.0
...,...,...,...
495,12,202412032,14230.0
496,12,202412033,2500.0
497,12,202412034,18591.0
498,12,202412036,27730.0


In [41]:
media = agrupado.groupby('Mes')['Monto_PI'].mean().reset_index()
media

,Mes,Monto_PI
0,01,7998.300000
1,02,13164.570000
2,03,12147.180000
3,04,11277.649123
4,05,13258.604359
5,06,13094.563636
6,07,9794.119048
7,08,11152.675000
8,09,12320.202034
9,10,12953.191525


In [42]:
moda = agrupado.groupby('Mes')['Monto_PI'].apply(lambda x: x.mode().iloc[0]).reset_index()
moda

,Mes,Monto_PI
0,01,1460.0
1,02,1252.0
2,03,116.0
3,04,81.0
4,05,364.0
5,06,142.0
6,07,18.0
7,08,265.0
8,09,35.0
9,10,470.0


In [43]:
mediana = agrupado.groupby('Mes')['Monto_PI'].median().reset_index()
mediana

,Mes,Monto_PI
0,01,6809.0
1,02,7488.0
2,03,8544.0
3,04,9708.0
4,05,10861.0
5,06,8819.0
6,07,8166.0
7,08,7330.0
8,09,11368.0
9,10,8626.0


In [46]:
comparacion = pd.merge(media, moda , on = 'Mes' , suffixes=('_media', '_moda'))
comparacion_final = pd.merge(comparacion, mediana, on ='Mes', suffixes=('', '_mediana'))
comparacion_final


,Mes,Monto_PI_media,Monto_PI_moda,Monto_PI
0,01,7998.300000,1460.0,6809.0
1,02,13164.570000,1252.0,7488.0
2,03,12147.180000,116.0,8544.0
3,04,11277.649123,81.0,9708.0
4,05,13258.604359,364.0,10861.0
5,06,13094.563636,142.0,8819.0
6,07,9794.119048,18.0,8166.0
7,08,11152.675000,265.0,7330.0
8,09,12320.202034,35.0,11368.0
9,10,12953.191525,470.0,8626.0
